## UseLib

In [25]:
############## 전처리 및 데이터 핸들링 ###############################

import numpy as np
import  pandas as pd
import os
from konlpy.tag import Okt 
okt = Okt()
from tqdm.notebook import tqdm
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from tqdm import tqdm
import time

############## 머신러닝 모델#########################
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.svm import SVC
svc =  SVC()
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
import xgboost as xgb
from xgboost import XGBClassifier
xgc = XGBClassifier()


############## 평가 ###############################
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

## UseData

In [2]:
cd ~/WorkSpace/GoogleDrive/제주_전기차/data 

/home/miwoos/WorkSpace/GoogleDrive/제주_전기차/data


In [3]:
ls ## 데이터 위치 파악 및 데이터 리스트 확인

 elc_usewordcloud.png              빈도수파악.png
 elecafe.csv                      '전기차 동호회 전기차크롤링.csv'
 lr.pkl                            전기차_전처리및토큰화.csv
 lrc.pkl                           전기차관련검색어인기순.csv
 model/                            전기차관심도지역.csv
 tvect.pkl                         전기차관심증가.csv
 감성사전분석.csv                  전기차전처리추가.csv
 관련키워드.csv                    전기차키워드관련주제인기순.csv
 네이버-제주전기차후기통합.csv     전체본문의댓글결과_2022.03.06.xlsx
 네이버블로그감성분석비교.csv      제사모크롤링.csv
 느영나영전기차크롤링.csv          카페통합감성추가.csv
 느영나영전기차크롤링.csv.part     투플럭스sjeju.csv
'본문 검색 결과_2022.03.06.xlsx'


In [4]:
df_train= pd.read_csv('전기차_전처리및토큰화.csv')
df_cafe = pd.read_csv('elecafe.csv')
df_Blog = pd.read_csv('네이버-제주전기차후기통합.csv')

## 러닝

### 널값 제거 및 전처리

In [5]:
df_train.head(2)

,전처리,평가
0,아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...,P
1,현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...,P


In [6]:
print(df_train.isnull().sum())
print(df_cafe.isnull().sum())
print(df_Blog.isnull().sum())

전처리    42
평가      0
dtype: int64
제목      0
내용      0
댓글      0
유형      0
조회수     0
작성날짜    0
월별      0
카페명     0
년별      0
dtype: int64
제목          0
날짜          0
내용          0
href        0
index    6096
dtype: int64


In [7]:
del df_Blog['index'] # 인덱스칼럼이 널값이많아서 컬럼 자체를 없애 버림 
df_train.dropna(inplace=True) # 드랍으로 널값제거

### 이진분류

In [8]:
df_이진분류 = copy.deepcopy(df_train[df_train !='E'])

In [9]:
df_이진분류.head(2)
df_이진분류.dropna(inplace=True)
df_이진분류.isnull().sum()

전처리    0
평가     0
dtype: int64

In [10]:
X_train,X_test,y_train,y_test = train_test_split (
    df_이진분류['전처리'],df_이진분류['평가'],test_size=0.25,random_state=2021
)

In [11]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((218401,), (72801,), (218401,), (72801,))

##### 타겟값 라벨링

In [12]:
y_train_label = LabelEncoder().fit_transform(y_train)
y_test_label = LabelEncoder().fit_transform(y_test)

In [13]:
y_train_label,y_test_label

(array([1, 0, 1, ..., 0, 1, 0]), array([1, 1, 0, ..., 1, 1, 1]))

#### CV binary

In [14]:
cvect_binary =  CountVectorizer()
cvect_binary.fit(X_train)

CountVectorizer()

In [15]:
X_train_cv_binary = cvect_binary.transform(X_train)
X_test_cv_binary =  cvect_binary.transform(X_test)

#### TF-IDF binary

In [16]:
tvect_binary = TfidfVectorizer(ngram_range=(1,2), max_df = 0.9)

In [17]:
%time tvect_binary.fit(X_train)
%time X_train_tv_binary = tvect_binary.transform(X_train)
%time X_test_tv_binary = tvect_binary.transform(X_test)

CPU times: user 2min 54s, sys: 916 ms, total: 2min 55s
Wall time: 2min 54s
CPU times: user 16.4 s, sys: 124 ms, total: 16.5 s
Wall time: 16.5 s
CPU times: user 5.2 s, sys: 16 ms, total: 5.22 s
Wall time: 5.22 s


### 다중분류

In [18]:
XX_train,XX_test,yy_train,yy_test = train_test_split (
    df_train['전처리'],df_train['평가'],test_size=0.25,random_state=2021
)

In [19]:
XX_train.shape,XX_test.shape,yy_train.shape,yy_test.shape

((296083,), (98695,), (296083,), (98695,))

#### cv 

In [20]:
cvect  = CountVectorizer(ngram_range=(1,2), max_df = 0.9)
cvect.fit(XX_train)

CountVectorizer(max_df=0.9, ngram_range=(1, 2))

In [21]:
XX_train_cv = cvect.transform(XX_train)
XX_test_cv=  cvect.transform(XX_test)

#### TF-IDF

In [22]:
tvect = TfidfVectorizer(ngram_range=(1,2), max_df = 0.9)

In [23]:
tvect.fit(XX_train)
XX_train_tv = tvect.transform(XX_train)
XX_test_tv = tvect.transform(XX_test)


## 모델 비교 평가

In [ ]:
XX_test_cv

In [24]:
def auto(model,train_test):
    model.fit(train_test[0],y_train_label)
    pred=model.predict(train_test[1])
    if train_test[1] in [X_test_cv_binary,X_test_tv_binary]:
        ave = 'binary'
    else:
        ave = 'macro'
    accuracy_score(y_test_label, pred)
    p = precision_score(y_test_label, pred,average=ave)
    r = recall_score(y_test_label, pred,average=ave)
    f1 = f1_score(y_test_label, pred,average=ave)
    return f'스코어:{model.score(train_test[1],y_test_label)}\nP:{p}\nR:{r}\nF1:{f1}'

In [ ]:
results = []
# !mkdir model
g = globals()
# for model in [lr,nb,svc,knn,rfc]:
for model in tqdm([svc,knn,rfc]):
    for train_test in tqdm([[X_train_cv_binary,X_test_cv_binary],[X_train_tv_binary,X_test_tv_binary]]):
        print(f'모델:{model}\n벡터:{train_test}일때\n{auto(model,train_test)}입니다.')
        results.append(f'모델:{model}\n벡터:{test}일때\n{auto(model,train_test)}입니다.')
        joblib.dump(model,f'model/전기차이진{model}.pkl')
    # for train_test in [[XX_train_cv,XX_test_cv],[XX_train_tv,XX_test_tv]]:
    #     print(f'다중분류모델모델:{model}\n,벡터:{test}일때\n{auto(model,train_test)}입니다.')
    #     results.append(f'다중분류모델:{model}\n벡터:{test}일때\n{auto(model,train_test)}입니다.')
    #     joblib.dump(model,f'model/전기차삼진{model}.pkl')



  0%|                                                     | 0/2 [00:00<?, ?it/s]

In [ ]:
# def auto2(model,vect,i):
#     g= globals()
#     if vect == 'cvect':
#         g[f'{vect}{i}'] = CountVectorizer(ngram_range=(1,i))
#     elif vect == 'tvect':
#         g[f'{vect}{i}'] = TfidfVectorizer(ngram_range=(1,i))
#     g[f'{vect}{i}'].fit(X_train)
#     g[f'XX_train_{vect}{i}'] = g[f'{vect}{i}'].transform(XX_train)
#     g[f'XX_test_{vect}{i}'] = g[f'{vect}{i}'].transform(XX_test)
#     model.fit(g[f'XX_train_{vect}{i}'],y_train_label)
#     return model.score(g[f'XX_test_{vect}{i}'],y_test_label)

In [ ]:
# !mkdir model
# for model in [lr,nb,xgc,svc,knn,rfc]:
#     for vect in ['cvect','tvect']:
#             for i in [1,2]:
#                     print(f'다중분류 모델:{model}\t,벡터:{vect}\tngram_range:{i}일때:{auto2(model,vect,i)}입니다.')
#                     joblib.dump(model,f'다중분류model/전기차{vect}{model}ngram{i}.pkl')


In [ ]:
for model in [lr,nb,svc,knn,rfc]:
    for test in [X_test_cv_binary,X_test_tv_binary,XX_test_tv,XX_test_tv]:
        pred=model.predict(test)
        if test in [X_test_cv_binary,X_test_tv_binary]:
            ave = 'binary'
        else:
            ave = 'macro'
        accuracy_score(y_test_label, pred)
        p = precision_score(y_test_label, pred,average=ave)
        print(f'{model}일떄+{test}이면+{p}는p입니다')
        r = recall_score(y_test_label, pred,average=ave)
        print(f'{model}일떄+{test}이면+{r}는r입니다')
        f1 = f1_score(y_test_label, pred,average=ave)
        print(f'{model}일떄+{test}이면+{f1}는f1입니다')


#### 모델저장

In [ ]:
# !mkdir model
# for model in [lr,nb,xgc,svc,knn,rfc]:
#     joblib.dump(model,f'model/전기차{model}.pkl')
for vect in [cvect_binary,tvect_binary,cvect,tvect]:
    joblib.dump(vect,f'model/전기차{vect}.pkl')